In [69]:
# Boilerplate at top of file
%load_ext autoreload
%load_ext rmagic
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime
import math

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
from ggplot import *


### Custom
import nltk
import sys
sys.path.append('../')
import cleaner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [66]:
df = pd.read_csv('../data/train_sample.csv')
# df_full = pd.read_csv('../data/train.csv.csv')

# Feature explore

In [41]:
### Util Functions
def tokenize_string(string):
    string = str(string)
    return string.lower().strip().split(' ')

def product_has_brand(row):
    BRAND_KEY = 'Brand name'.lower()
    attributes = eval(row['attributes'])
    search_tokens = set(tokenize_string(row['search_term']))
    for attr in attributes:
        if attr[0].lower().find(BRAND_KEY) != -1:
            return True
    return False
def product_searched_brand(row):
    BRAND_KEY = 'Brand name'.lower()
    attributes = eval(row['attributes'])
    search_tokens = set(tokenize_string(row['search_term']))
    for attr in attributes:
        if attr[0].lower().find(BRAND_KEY) != -1:
            attr_tokens = set(tokenize_string(attr[1]))
            return len(attr_tokens.intersection(search_tokens))
    return 0
                
            


In [42]:
df_feats = pd.DataFrame()
# df_feats['match_search'] = df.apply(match_search_term_to_title,axis=1)
# df_feats['match_desc'] = df.apply(match_search_term_to_description,axis=1)
# df_feats['desc_count'] = df.apply(count_description,axis=1)
# df_feats['search_desc_count'] = df.apply(count_search_description_occurences,axis=1)
# df_feats['search_desc_match'] = df.apply(match_search_description_occurences,axis=1)
df_feats['prod_has_brand'] = df.apply(product_has_brand,axis=1)
df_feats['prod_search_brand_match'] = df.apply(product_searched_brand,axis=1)
df_feats['relevance'] = df['relevance']

In [46]:
df[df_feats['prod_has_brand']]

,index,id,product_uid,product_title,search_term,relevance,product_uid.1,product_description,attributes
22,16545,51037,113077,Hedrix 11 oz. Match of PPU8-7 Chamois Tan Glos...,chamois,2.67,113077,The Empire Line Levels Set (2-Pack) can be use...,"[('MFG Brand Name', 'Alexandria Moulding')]"
33,22857,70354,119732,Elanti Oval Vessel Bathroom Sink in White,vessel bowls,3.00,119732,Simpson Strong-Tie connectors make projects ea...,"[('MFG Brand Name', 'DEWALT')]"
46,30527,93401,128679,Brinkmann 18 lb. Apple Wood Chunks,brinkmann smoker,2.00,128679,NaN,"[('MFG Brand Name', 'Southern Enterprises')]"
51,36601,111925,136737,BEHR Premium Plus Ultra 1-gal. #BL-W13 Silver ...,silver polish,2.67,136737,NaN,"[('MFG Brand Name', 'Rubi')]"


In [68]:
# show the attributes
set(reduce(lambda l,r: l + r, df['attributes'].apply(lambda x: [atr[0] for atr in eval(x)]).tolist()))

{'Accessory Type',
 'Application Method',
 'Assembled Height (in.)',
 'Assembled Width (in.)',
 'Assembly Required',
 'Average Life (hours)',
 'Bath Faucet Type',
 'Battery Amp Hours',
 'Brand compatibility',
 'Bullet01',
 'Bullet02',
 'Bullet03',
 'Bullet04',
 'Bullet05',
 'Bullet06',
 'Bullet07',
 'Bullet10',
 'Bullet14',
 'Bullet15',
 'Case Included',
 'Certifications and Listings',
 'Color Family',
 'Commercial / Residential',
 'Connection 1',
 'Convertible to post mount',
 'Door Type',
 'Duct Type',
 'ENERGY STAR Certified',
 'Electronic',
 'Element No.6 Size (In.)',
 'Exact Width x Depth',
 'Features',
 'Fitting 1 size',
 'Flushing Type',
 'Head Pressure (ft.)',
 'Ice Maker',
 'Ionizing',
 'Kitchen Faucet Type',
 'Light Source',
 'MFG Brand Name',
 'Material',
 'Maximum fastener size (in.)',
 'Miter Saw Type',
 'Nominal panel height (ft.)',
 'Number in Package',
 'Number of USB Ports',
 'Outlet Size',
 'Patching & Repair Product Type',
 'Power Tool Product Type',
 'Primed',
 'Pro